In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime

In [2]:
#Cargando el CSV en un dataframe
df = pd.read_csv("test_data.csv", delimiter=",")
df.head(5)

,year,month,day_of_month,quarter_hour,campaign,category,disposition
0,2017,December,1,NaN,prospecting,NaN,Email
1,2017,December,1,NaN,outbound,NaN,Email
2,2017,December,1,NaN,prospecting,NaN,Email
3,2017,December,1,NaN,prospecting,NaN,Email
4,2017,December,1,NaN,prospecting,NaN,Email


## Completando el campo vacío en quarter_hour

In [3]:
#Revisando cuantos registros tienen vacío el campo quarter_hour
df_test = df[df['quarter_hour'].isnull() == True]
df_test.shape[0] #1838 campos vacíos

1838

In [4]:
#Revisando los diferentes valores que tiene el campo quarter_hour, y agrupando para contarlos
df_test = df.groupby(['quarter_hour']).size().reset_index(name='counts')
df_test.head(5)

,quarter_hour,counts
0,10:00,39
1,10:15,44
2,10:30,48
3,10:45,48
4,11:00,48


In [5]:
#Llenando los quarter_hour con 00:00
df['quarter_hour'] = df.quarter_hour.fillna('00:00')
df.head(5)

,year,month,day_of_month,quarter_hour,campaign,category,disposition
0,2017,December,1,00:00,prospecting,NaN,Email
1,2017,December,1,00:00,outbound,NaN,Email
2,2017,December,1,00:00,prospecting,NaN,Email
3,2017,December,1,00:00,prospecting,NaN,Email
4,2017,December,1,00:00,prospecting,NaN,Email


In [6]:
#Revisando cuantos registros tienen vacío el campo quarter_hour
df_test = df[df['quarter_hour'].isnull() == True]
df_test.shape[0] #ningun campo vacío

0

In [7]:
#Revisando cuantos registros quedaron con 00:00
df_test = df[df['quarter_hour'] == '00:00']
df_test.shape[0] #1838 campos

1838

## Creando los timestamps

In [8]:
#Concatenando toda la fecha en un sólo campo 
df['fulldate'] = df['year'].map(str) +'-'+ df['month'] +'-'+ df['day_of_month'].map(str) +' '+ df['quarter_hour']
df['fulldate'] = df['fulldate'].apply(lambda x: dt.datetime.strptime(x,'%Y-%B-%d %H:%M'))
df.head(5)

,year,month,day_of_month,quarter_hour,campaign,category,disposition,fulldate
0,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01
1,2017,December,1,00:00,outbound,NaN,Email,2017-12-01
2,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01
3,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01
4,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01


In [9]:
def converter_MST(row):
    return row['fulldate'].tz_localize('US/Mountain').tz_convert('US/Mountain')

#Creando la fecha con Timezone MST
df['MST_Time'] = df.apply(converter_MST, axis=1)
df.head(5)

,year,month,day_of_month,quarter_hour,campaign,category,disposition,fulldate,MST_Time
0,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01,2017-12-01 00:00:00-07:00
1,2017,December,1,00:00,outbound,NaN,Email,2017-12-01,2017-12-01 00:00:00-07:00
2,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01,2017-12-01 00:00:00-07:00
3,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01,2017-12-01 00:00:00-07:00
4,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01,2017-12-01 00:00:00-07:00


In [10]:
def converter_UTC(row):
    return row['fulldate'].tz_localize('US/Mountain').tz_convert('UTC')

#Creando la fecha con Timezone UTC
df['UTC_Time'] = df.apply(converter_UTC, axis=1)

df.head(5)

,year,month,day_of_month,quarter_hour,campaign,category,disposition,fulldate,MST_Time,UTC_Time
0,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01,2017-12-01 00:00:00-07:00,2017-12-01 07:00:00+00:00
1,2017,December,1,00:00,outbound,NaN,Email,2017-12-01,2017-12-01 00:00:00-07:00,2017-12-01 07:00:00+00:00
2,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01,2017-12-01 00:00:00-07:00,2017-12-01 07:00:00+00:00
3,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01,2017-12-01 00:00:00-07:00,2017-12-01 07:00:00+00:00
4,2017,December,1,00:00,prospecting,NaN,Email,2017-12-01,2017-12-01 00:00:00-07:00,2017-12-01 07:00:00+00:00


In [11]:
#Verificando los tipos de datos
df.dtypes

year                                  int64
month                                object
day_of_month                          int64
quarter_hour                         object
campaign                             object
category                             object
disposition                          object
fulldate                     datetime64[ns]
MST_Time        datetime64[ns, US/Mountain]
UTC_Time                datetime64[ns, UTC]
dtype: object